In [4]:
import os

def process_labels(labels_file, output_dir):
    """
    Process the labels.txt file and create individual label files for each image
    with only selected parameters (class number, class name, bounding box coordinates).
    
    :param labels_file: Path to the labels.txt file.
    :param output_dir: Directory to save the individual label files.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read the labels file
    with open(labels_file, 'r') as file:
        lines = file.readlines()
    
    # Group labels by image index
    grouped_labels = {}
    for line in lines:
        parts = line.strip().split()
        image_index = int(parts[0])  # Extract the image index (first value in the line)
        class_number = parts[1]
        class_name = parts[2]
        bbox_coords = parts[6:10]  # Bounding box coordinates: x_min, y_min, x_max, y_max
        
        # Combine into the desired format
        filtered_label = f"{class_number} {class_name} {' '.join(bbox_coords)}"
        
        if image_index not in grouped_labels:
            grouped_labels[image_index] = []
        grouped_labels[image_index].append(filtered_label)
    
    # Write individual label files
    for image_index, labels in grouped_labels.items():
        filename = f"{image_index:06d}.txt"  # Create a filename like 000000.txt
        output_path = os.path.join(output_dir, filename)
        with open(output_path, 'w') as output_file:
            output_file.write("\n".join(labels))
    
    print(f"Processed labels and saved them to {output_dir}")

# Example usage
labels_file = "valid1/labels1.txt"  # Replace with the actual path to your labels.txt file
output_dir = "valid1/labels"  # Replace with the desired output directory
process_labels(labels_file, output_dir)

Processed labels and saved them to valid1/labels


In [5]:
def process_labels(labels_file, output_dir):
    """
    Process the labels.txt file and append labels to existing label files if they already exist.
    Handles image filenames with 10-digit zero-padded numbers.
    
    :param labels_file: Path to the labels.txt file.
    :param output_dir: Directory containing the existing label files.
    """
    # Read the labels file
    with open(labels_file, 'r') as file:
        lines = file.readlines()
    
    # Group labels by image index
    grouped_labels = {}
    for line in lines:
        parts = line.strip().split()
        image_index = int(parts[0])  # Extract the image index (first value in the line)
        class_number = parts[1]
        class_name = parts[2]
        bbox_coords = parts[6:10]  # Bounding box coordinates: x_min, y_min, x_max, y_max
        
        # Combine into the desired format
        filtered_label = f"{class_number} {class_name} {' '.join(bbox_coords)}"
        
        if image_index not in grouped_labels:
            grouped_labels[image_index] = []
        grouped_labels[image_index].append(filtered_label)
    
    # Append to or create label files
    for image_index, labels in grouped_labels.items():
        filename = f"{image_index:010d}.txt"  # Filename like 0000000001.txt
        output_path = os.path.join(output_dir, filename)
        
        # Append to the file if it exists; otherwise, create a new one
        with open(output_path, 'a') as output_file:  # Use 'a' mode to append
            output_file.write("\n".join(labels) + "\n")
    
    print(f"Processed labels and appended them to {output_dir}")

# Example usage
labels_file = "valid1/labels2.txt"  # Replace with the actual path to your labels.txt file
output_dir = "valid1/labels"  # Replace with the directory containing existing label files
process_labels(labels_file, output_dir)

Processed labels and appended them to valid1/labels


In [6]:
import os

def update_class_numbers(label_dir):
    """
    Update class numbers in existing label files based on the class name.
    
    :param label_dir: Directory containing the label files to update.
    """
    # Class mapping: Class name -> Class number
    class_mapping = {
        "Car": "0",
        "Cyclist": "1",
        "Pedestrian": "2"
    }
    
    # Iterate over all files in the directory
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(label_dir, filename)
            
            # Read the content of the file
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            # Update class numbers
            updated_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 6:
                    continue  # Skip lines that don't have enough data
                current_class_number = parts[0]
                class_name = parts[1]
                
                # Assign the correct class number
                if class_name in class_mapping:
                    correct_class_number = class_mapping[class_name]
                    parts[0] = correct_class_number  # Update the class number
                    updated_lines.append(" ".join(parts))
                else:
                    # Keep the line unchanged if class name is not in the mapping
                    updated_lines.append(line.strip())
            
            # Overwrite the file with updated content
            with open(file_path, 'w') as file:
                file.write("\n".join(updated_lines) + "\n")
    
    print(f"Updated class numbers in all label files in {label_dir}")

# Example usage
label_dir = "valid1/labels"  # Replace with the directory containing your label files
update_class_numbers(label_dir)

Updated class numbers in all label files in valid1/labels


In [8]:
def normalize_labels(label_dir, image_dir, output_dir, class_mapping):
    """
    Normalize labels to YOLO format: <class_id> <cx> <cy> <w> <h>.
    
    :param label_dir: Directory containing raw label files.
    :param image_dir: Directory containing corresponding images (for dimensions).
    :param output_dir: Directory to save normalized labels.
    :param class_mapping: Dictionary mapping class names to IDs.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            label_path = os.path.join(label_dir, filename)
            image_path = os.path.join(image_dir, filename.replace(".txt", ".png"))  # Adjust if using another extension
            output_path = os.path.join(output_dir, filename)
            
            # Check if corresponding image exists
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_path} not found for {label_path}. Skipping.")
                continue
            
            # Get image dimensions
            try:
                from PIL import Image
                with Image.open(image_path) as img:
                    img_width, img_height = img.size
            except Exception as e:
                print(f"Error: Unable to open image {image_path}. {e}")
                continue
            
            # Read and process label file
            with open(label_path, 'r') as file:
                lines = file.readlines()
            
            normalized_labels = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 6:
                    print(f"Skipping malformed line in {label_path}: {line.strip()}")
                    continue
                
                try:
                    # Extract class ID and bounding box coordinates
                    class_name = parts[1]
                    if class_name not in class_mapping:
                        print(f"Skipping unknown class '{class_name}' in {label_path}: {line.strip()}")
                        continue
                    
                    class_id = class_mapping[class_name]
                    x_min, y_min, x_max, y_max = map(float, parts[2:6])
                    
                    # Normalize bounding box
                    x_center = ((x_min + x_max) / 2) / img_width
                    y_center = ((y_min + y_max) / 2) / img_height
                    width = (x_max - x_min) / img_width
                    height = (y_max - y_min) / img_height
                    
                    # Append to normalized labels
                    normalized_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                except ValueError as e:
                    print(f"Error processing line in {label_path}: {line.strip()} ({e})")
            
            # Write normalized labels to output file
            if normalized_labels:
                with open(output_path, 'w') as file:
                    file.write("\n".join(normalized_labels) + "\n")
                print(f"Normalized labels for {label_path} -> {output_path}")
            else:
                print(f"No valid labels found in {label_path}. File will be left empty.")

# Example usage
label_dir = "valid1/labels"  # Directory with raw labels
image_dir = "valid1/images"  # Directory with corresponding images
output_dir = "valid1/labels"  # Directory to save normalized labels
class_mapping = {
    "Car": "0",
    "Cyclist": "1",
    "Pedestrian": "2"
}

normalize_labels(label_dir, image_dir, output_dir, class_mapping)

Normalized labels for valid1/labels\000000.txt -> valid1/labels\000000.txt
Normalized labels for valid1/labels\0000000000.txt -> valid1/labels\0000000000.txt
Normalized labels for valid1/labels\0000000001.txt -> valid1/labels\0000000001.txt
Normalized labels for valid1/labels\0000000002.txt -> valid1/labels\0000000002.txt
Normalized labels for valid1/labels\0000000003.txt -> valid1/labels\0000000003.txt
Normalized labels for valid1/labels\0000000004.txt -> valid1/labels\0000000004.txt
Normalized labels for valid1/labels\0000000005.txt -> valid1/labels\0000000005.txt
Normalized labels for valid1/labels\0000000006.txt -> valid1/labels\0000000006.txt
Normalized labels for valid1/labels\0000000007.txt -> valid1/labels\0000000007.txt
Normalized labels for valid1/labels\0000000008.txt -> valid1/labels\0000000008.txt
Normalized labels for valid1/labels\0000000009.txt -> valid1/labels\0000000009.txt
Normalized labels for valid1/labels\0000000010.txt -> valid1/labels\0000000010.txt
Normalized l